In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Teeth DataSet.zip','r') as zip:
  zip.extractall()

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import cv2

In [5]:

# Hyperparameters
img_height, img_width = (224, 224)
batch_size = 32
epochs = 40
learning_rate = 0.001

# Data paths
train_dir = 'Teeth_Dataset/Training'
val_dir = 'Teeth_Dataset/Validation'
test_dir = 'Teeth_Dataset/Testing'
# def preprocess_image(image):
#     image = tf.image.central_crop(image, central_fraction=0.875)  # Center crop
#     image = tf.image.resize(image, [224, 224])  # Resize to target size
#     return image
# Data augmentation
datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_image,
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range= 0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = datagen.flow_from_directory(train_dir,target_size=(224,224),
                                              batch_size=4,class_mode ='categorical',subset='training')
validation_generator = datagen.flow_from_directory(val_dir,target_size=(224,224),
                                              batch_size=4,class_mode ='categorical',subset='validation')

Found 2471 images belonging to 7 classes.
Found 204 images belonging to 7 classes.


In [6]:
# VGG 16 Model
from tensorflow.keras.applications import VGG16
model_vgg16 = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')
model_vgg16.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Load the pre-trained VGG16 model (include weights if needed)
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.regularizers import l2

input_shape = (224 , 224, 3)  # Replace with the actual shape if different

# Create the input layer
inputs = Input(shape=input_shape)

# Pass the input to the pre-trained model (frozen layers)
x = model_vgg16(inputs)
# x = Dropout(0.5)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
# x = Dropout(0.75)(x)
outputs = Dense(7, activation='softmax')(x)

# Create the final model
transfer_model = Model(inputs=inputs, outputs=outputs)

transfer_model.layers[1].trainable = False


transfer_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 27,563,847 (105.15 MB)

 Trainable params: 12,849,159 (49.02 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [8]:
transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
history=transfer_model.fit(train_generator, epochs=20, validation_data=validation_generator)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


618/618 ━━━━━━━━━━━━━━━━━━━━ 52s 74ms/step - accuracy: 0.3566 - loss: 2.7646 - val_accuracy: 0.4069 - val_loss: 1.5466
Epoch 2/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.5538 - loss: 1.2091 - val_accuracy: 0.4853 - val_loss: 1.6786
Epoch 3/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 39s 63ms/step - accuracy: 0.6354 - loss: 1.0234 - val_accuracy: 0.4853 - val_loss: 1.5057
Epoch 4/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.6964 - loss: 0.8475 - val_accuracy: 0.4755 - val_loss: 1.5839
Epoch 5/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 81s 64ms/step - accuracy: 0.7166 - loss: 0.8106 - val_accuracy: 0.5049 - val_loss: 1.4746
Epoch 6/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 40s 65ms/step - accuracy: 0.7586 - loss: 0.6902 - val_accuracy: 0.5637 - val_loss: 1.5556
Epoch 7/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.7525 - loss: 0.7050 - val_accuracy: 0.4657 - val_loss: 1.5983
Epoch 8/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.7987 - loss: 0.5588 - val_accurac

In [ ]:
teeth =transfer_model.save('teeth_class_VGG16.h5')

Try ResNet

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=7,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

# Add an InputLayer to define the input shape
resnet_model.add(InputLayer(input_shape=(224, 224, 3)))
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(7, activation='softmax'))

resnet_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,640,391 (94.00 MB)

 Trainable params: 1,052,679 (4.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [16]:
resnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
history=resnet_model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 58s 76ms/step - accuracy: 0.1497 - loss: 2.1491 - val_accuracy: 0.1471 - val_loss: 1.9221
Epoch 2/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.1973 - loss: 1.9146 - val_accuracy: 0.2010 - val_loss: 1.9224
Epoch 3/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.2194 - loss: 1.8650 - val_accuracy: 0.2402 - val_loss: 1.9147
Epoch 4/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 81s 64ms/step - accuracy: 0.2208 - loss: 1.8487 - val_accuracy: 0.2843 - val_loss: 1.9113
Epoch 5/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 41s 67ms/step - accuracy: 0.2424 - loss: 1.8341 - val_accuracy: 0.2500 - val_loss: 1.9401
Epoch 6/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 83s 68ms/step - accuracy: 0.2402 - loss: 1.8393 - val_accuracy: 0.2304 - val_loss: 1.9386
Epoch 7/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.2348 - loss: 1.8363 - val_accuracy: 0.2696 - val_loss: 1.9261
Epoch 8/10
618/618 ━━━━━━━━━━━━━━━━━━━━ 39s 63ms/step - accuracy: 0.2309 - loss: 1.8260 - 

In [18]:
teeth_ResNet =resnet_model.save('teeth_class_ResNet.h5')

In [24]:
import cv2

img = cv2.imread('/content/Teeth_Dataset/Training/CaS/a_100_0_1462.jpg')
img = cv2.resize(img, (224,224))
img = np.reshape(img, [1,224,224,3])
img = img/255

preds = transfer_model.predict(img)

max_idx = np.argmax(preds)
class_indices = {v:k for k,v in train_generator.class_indices.items()}
class_indices[max_idx]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


'CaS'

In [20]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [21]:
%%writefile app.py
import streamlit as st

st.set_option('deprecation.showfileUploaderEncoding', False)
@st.cache(allow_output_mutation=True) # make the func loaded once
def load_model():
  model=tf.keras.models.load_model('teeth_classifier.hdf5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Teeth Classification
         """
         )

file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):

        size = (224,224)
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.

        img_reshape = img[np.newaxis,...]

        prediction = model.predict(img_reshape)

        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    class_names = ['CaS','CoS','Gum','MC','OC','OLP','OT']
    string = "This image most likely is: "+ class_names[np.argmax(predictions)]
    st.success(string)


Writing app.py


In [31]:
!pip install pyngrok

In [27]:
from pyngrok import ngrok
!ngrok authtoken 2lHI4EpeCm3Ch6JAicGlvAjIxQa_cHobUaM1FE7nMePg2RVd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [32]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.93.196:8501

  Stopping...
  Stopping...


In [37]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [38]:
from pyngrok import ngrok

# Define the tunnel configuration
tunnel_config = {
    "addr": 8501,  # The port your Streamlit app is running on
    "proto": "http"  # The protocol to use (http or tcp)
}

# Connect with the specified configuration
url = ngrok.connect(**tunnel_config)
print(url)

NgrokTunnel: "https://8219-34-125-93-196.ngrok-free.app" -> "http://localhost:8501"


In [39]:
!cat /content/nohup.out




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.93.196:8501

